Reference: https://makersportal.com/blog/2018/9/17/audio-processing-in-python-part-ii-exploring-windowing-sound-pressure-levels-and-a-weighting-using-an-iphone-x

# Audio Processing

# Proper Windowing Technique

In the previous tutorial, I discussed sampling and the Nyquist frequency. Henceforth, I will be acting under the assumption that our sample rate is above the Nyquist frequency. With that in mind, I wanted to discuss another limitation of recording data that is related to sampling, but is not part of the Nyquist theorem. When sampling data, we often want to sample over short periods so that our recording device, computer memory, and processing power don’t get overwhelmed. This short time sampling is called windowing. Windowing is done for a multitude of reasons - most of them having to do with saving resources and reducing error in measurements. I will not cover windowing in great detail here, but I will discuss its consequences to signal processing. In our case here, we will only be using a simple rectangular window, which means that we will be taking chunks of data and calculating the frequency spectrum from the raw data. Other types of windows (Hanning, Hamming, Welch, Blackman, etc.) solve issues resulting from taking data in chunks, so they can be incredibly powerful. 

The most significant artifact of windowing is its limitation on resolving natural frequencies. When windowing a signal, the length of the window determines the minimum frequency resolvable in the system. For example, if we are sampling a 100Hz sine wave, the minimum window length needed to resolve the 100 Hz wave is 0.01 s (1/100 s), which is the period of the wave. This can be difficult, especially when analyzing lower frequencies because sometimes long periods of data is not possible. In audio, we are typically interested in the 20 Hz - 20 kHz range, so the minimum window period is 10 ms. 

it is often recommended to use a sample window of about 5x the lowest natural frequency period. In the case of the 100 Hz sine wave, we should use a window 50 ms long.

Windowing technique is import when sampling data because of its relation to the natural frequencies of the system and also the reduction of noise. During our analysis, I will ensure that our window period is at least 5x the lowest predicted frequency. In acoustics, the lowest frequency can be assumed to be 20 Hz (sometimes 10 Hz), so we can assume a minimum window period of  250 ms. In cases like the range of the human voice, we see much shorter periods (25-50 ms) because the lowest frequency for a human voice is between 80-180 Hz , but the range of vocal analysis assumes an average low frequency of 200 Hz to emulate the time and frequency response of the human ear. In our case, I’ll be using 8192 points at 44.1 kHz - which gives us a window period of about 185 ms, which is sufficient for the low audible frequencies (minimum 27 Hz)

To learn more about windowing- work through the windowing tutorial in your notebooks folder.

# Frequency Spectrum of Real Data Recorded with pyaudio

Below is the routine for recording audio and taking the FFT of the signal. I used an iPhone to generate known sine wave frequencies so that I could check the accuracy of the FFT algorithm. The app I used is the ‘Signal Gen’ app on the iTunes App Store. The app allows you to choose the amplitude and frequency of the generated sine wave. It is a powerful tool for testing and validating the FFT windowing accuracy. If you are using an android phone, try the Frequency Generator app.

You will first need to make sure that you have all the python packages we use here downloaded. 

You can download all of these (except sounddevice) through your anaconda package manager. The packages will import below only if you have them downloaded through anaconda.

You will need to download sounddevice through a command window (you can use anaconda's prompt command window). In the window type the command:

conda install -c conda-forge python-sounddevice

(Say yes to any prompt questions)

In [ ]:
import pyaudio
import matplotlib.pyplot as plt
import numpy as np
import time
import sounddevice as sd

In [ ]:
sd.query_devices()

In [ ]:
form_1 = pyaudio.paInt16 # 16-bit resolution
chans = 1 # 1 channel
samp_rate = 48000 # 44.1kHz sampling rate
chunk = 8192 # 2^12 samples for buffer
dev_index = 1 # device index found by p.get_device_info_by_index(ii)

Now you can run your tests.  Once you have the signal generator app on your phone, and are sure your laptop is recording through either it's microphone or if you are using apple earbuds, through the microphone in the earbods. Place your phone near the microphone and play a single sinusoidal tone at 440 Hz. Once you have that playing constantly, run the next section to make a recording.

In [ ]:
audio = pyaudio.PyAudio() # create pyaudio instantiation

# create pyaudio stream
stream = audio.open(format = form_1,rate = samp_rate,channels = chans, \
                    input_device_index = dev_index,input = True, \
                    frames_per_buffer=chunk)

# record data chunk 
stream.start_stream()
data = np.fromstring(stream.read(chunk),dtype=np.int16)
stream.stop_stream()

Now you can create a frequency spectrum of your recording using the fast fourier transform. Run the next block of code. You should see a peak near 440 Hz.

In [ ]:
# compute FFT parameters
f_vec = samp_rate*np.arange((chunk/2))/chunk # frequency vector based on window size and sample rate
mic_low_freq = 5 # low frequency response of the mic (mine in this case is 5 Hz)
low_freq_loc = np.argmin(np.abs(f_vec-mic_low_freq))
fft_data = (np.abs(np.fft.fft(data))[0:int(np.floor(chunk/2))])/chunk
fft_data[1:] = 2*fft_data[1:]

max_loc = np.argmax(fft_data[low_freq_loc:])+low_freq_loc

# plot
plt.style.use('ggplot')
plt.rcParams['font.size']=18
fig = plt.figure(figsize=(13,8))
ax = fig.add_subplot(111)
plt.plot(f_vec,fft_data)
ax.set_ylim([0,2*np.max(fft_data)])
plt.xlabel('Frequency [Hz]')
plt.ylabel('Amplitude [Pa]')
ax.set_xscale('log')
plt.grid(True)

# max frequency resolution 
plt.annotate(r'$\Delta f_{max}$: %2.1f Hz' % (samp_rate/(2*chunk)),xy=(0.7,0.92),\
             xycoords='figure fraction')

# annotate peak frequency
annot = ax.annotate('Freq: %2.1f'%(f_vec[max_loc]),xy=(f_vec[max_loc],fft_data[max_loc]),\
                    xycoords='data',xytext=(0,30),textcoords='offset points',\
                    arrowprops=dict(arrowstyle="->"),ha='center',va='bottom')
    
plt.savefig('fft_1kHz_signal.png',dpi=300,facecolor='#FCFCFC')
plt.show()

The code above records 185 ms of data via pyaudio and analyzes the signal using the Fast Fourier Transform. We can find that be calculating 1/sampling rate = 1/44100, and multiply by the number of samples in our recorded chunk:

(1/44100)*8192 = 0.185795 seconds or 185 ms (rounded down).

Notice that the frequency peak is not exactly at 440 hz. Why do you think that is?  For an experiment, take not of the current Delta-f Max value in the top right corner. Now rerun the experiment changing the 'chunk' size to a different power of 2. Note that 8192=2^13, so try chunk = 2^12 or 2^14 or 2^15. What happens to the peak value near 440 Hz? What happens to the Delta-f Max value in the top right?

Delta-f Max is the maximum frequency resolution. The frequency resolution is defined as Fs/N in FFT. Where Fs is sample frequency, N is number of data points used in the FFT. Here N is equal to the 'chunk' size. 

So for the very first run of the code we have Fs = sample_rate/2, and a chunk size of 8192. That gives us (48000/2)/8192 = 2.9.

This is essentially the frequency bin size, much like a graph that has tick marks for each x,y increment. Some graphs tic marks increment by 1, other graphs have tic marks that increment by 5 etc.